# Aula 04 
## Controle de qualidade de dados para Data Science

## Agenda

* A importância do controle de qualidade dos dados em Data Science

* Outliers

* Técnicas mais eficazes para controle de qualidade de dados

* Exercício prático: preparação de dados

### Porque é importante o controle de qualidade dos dados usados em Data Science?

* Garbage-in, garbage-out (GIGO)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(r'/content/drive/My Drive/DadosPythonPro/garbage_in.csv')

In [ ]:
df.describe()

In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
df[df.values > 10000]

* Tempo computacional
* Qualidade da apresentação dos resultados

### Evitando "garbage" em Machine Learning

#### Dados imprecisos

#### Dados que refletem "preconceitos humanos"

#### Dados incompletos

#### Bias

#### Reflexos em:

* Treinamento
* Teste
* Estimativa

#### **Regra de ouro:** O analista de dados TEM QUE TER conhecimento sobre os dados que ele está analizando.

Quanto conhecimento?

Todo o conhecimento possível.

#### "Muitos problemas advém do fato do analista não conhecer os dados e também não conhecer as maneiras pelas quais eles falham". 
*Mikey Shulman, ider de machine-learning para a Kensho Technologies.*

#### "Causo" do inventário

### Outliers: técnicas para encontrar e remover

In [ ]:
import sklearn.datasets as sds
import pandas as pd

In [ ]:
boston = sds.load_boston()
print(type(boston))
x = boston.data
y = boston.target
columns = boston.feature_names

boston_df = pd.DataFrame(boston.data)
boston_df.columns = columns
print(boston_df.count())
boston_df.head()


Descrição: https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

* CRIM - taxa de criminalidade per capita por cidade
* ZN - proporção de terrenos residenciais divididos em lotes com mais de 25.000 pés quadrados.
* INDUS - proporção de acres não comerciais por cidade.
* CHAS - Variável fictícia Charles River (1 se o trecho limita o rio; 0 caso contrário)
* NOX - concentração de óxidos nítricos (partes por 10 milhões)
* RM - número médio de quartos por habitação
* AGE - proporção de unidades ocupadas pelo proprietário construídas antes de 1940
* DIS - distâncias ponderadas para cinco centros de emprego em Boston
* RAD - índice de acessibilidade às rodovias radiais
* TAX - taxa de imposto sobre a propriedade de valor total por \$10,000
* PTRATIO - proporção aluno-professor por cidade
* B - 1000(Bk - 0.63)^2 em que Bk é a proporção de negros por cidade
* LSTAT - /% menor status da população
* MEDV - Valor médio das casas ocupadas pelos proprietários em US $ 1000

**Método 1:** Boxplot

In [ ]:
import seaborn as sns
sns.boxplot(x=boston_df['DIS'])

**Método 2:** Análise de correlação

In [ ]:
boston_df.DIS.describe()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize= (10,10), dpi=100)
sns.heatmap(boston_df.corr())

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
ax.scatter(boston_df['INDUS'], boston_df['TAX'])
ax.set_xlabel('Proporção de acres não comerciais por cidade')
ax.set_ylabel(' Valor médio das casas ocupadas pelos proprietários em US $ 1000')
plt.show()

**Método 3:** Usando uma função matemática para descobrir outliers

**Teste Z (Z-Score)**

Teste estatístico que pressupõe que a distribuição dos valores se aproxima da Distribuição Normal.

**Distribuição Normal**

In [ ]:
import numpy as np
import scipy.stats as stats
import math

mu = 0
variance = 1
sigma = math.sqrt(variance)
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
plt.plot(x, stats.norm.pdf(x, mu, sigma))
plt.show()

#### Verificando se a distribuição é normal



In [ ]:
plt.hist(boston_df.TAX)

In [ ]:
from scipy import stats
z = np.abs(stats.zscore(boston_df))
print(z)

In [ ]:
threshold = 3
print(np.where(z > 3))

**Saída:** O primeiro array contém uma lista de linhas e o segundo array uma lista de colunas. Assim, z[55][1] tem um Z-score maior que três. Isso significa que, se considerarmos o valor do limiar = 3, esse dado é um outlier.

In [ ]:
z[55][1]

**Método 4:** Score IQR (Faixa interquartis)

*IQR = Q3 − Q1*

In [ ]:
Q1 = boston_df.quantile(0.25)
Q3 = boston_df.quantile(0.75)
IQR = Q3 - Q1
print(IQR)


#### Removendo os outliers

a) Encontrados pela distribuição

In [ ]:
print(boston_df.shape)
boston_df_o = boston_df[(z < 3).all(axis=1)]
print(boston_df_o.shape)

b: Encontrado pelo IQR

In [ ]:
boston_df_out = boston_df_o[~((boston_df_o < (Q1 - 1.5 * IQR)) |(boston_df_o > (Q3 + 1.5 * IQR))).any(axis=1)]
print(boston_df_out.shape)